# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [6]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [7]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [8]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [9]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [22]:
def foldl2(f, x0, lst):
    return foldr(lambda g, h: lambda x: h(f(x,g)), lambda x : x, lst)(x0)

foldl2(lambda x, y: x + y, 0, [1, 2, 3]), '---------', foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

(6, '---------', '123')

In [23]:
def foldr2(f, x0, lst):
    return foldl(lambda h, g: lambda x: h(f(g,x)), lambda x : x, lst)(x0)

foldr2(lambda x, y: x + y, 0, [1, 2, 3]), '---------', foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

(6, '---------', '123')

In [24]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3]), '---------', foldr2(f, 1, [1, 2, 3])

(0.16666666666666666, '---------', 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [52]:
def check_inv(a, b):
    if (set(sorted(list(a))).issubset(sorted(list(b)))) | (set(sorted(list(b))).issubset(sorted(list(a)))):
        return True
    else:
        return False

In [54]:
a = 'abcrotm'
b = 'tro'

check_inv(b, a)

True

In [56]:
a = 'asdfv'
b = 'cnga'

check_inv(a, b)

False

In [57]:
a = 'aangfh'
b = 'gaf'

check_inv(a, b)

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [20]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def iter_left(self):
        if self.left is not None:
            yield from self.left
        else: 
            yield self
    
    def iter_right(self):
        if self.right is not None:
            yield from self.right
        else:
            yield self
    
    def __iter__(self, direction='both'):
        if self.left is not None and self.right is not None:
            yield from self.left
            yield from self.right
        elif self.left is None and self.right is not None:
            yield from self.right
        elif self.left is not None and self.right is None:
            yield from self.left
        elif self.left is None and self.right is None:
            yield self                
    
    def __str__(self):
        return self.value
    
    def __repr__(self):
        return f'{self.value}'
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

In [21]:
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree)

[3, 4, 2]

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [24]:
def parse(expr):
    
    buf = []

    for val in expr.split(" "):
        if(val[0] == "("):
            buf.append("(")
            buf.append(val[1:])
        elif(val[-1] == ")"):
            buf.append(val[:-1])
            buf.append(")")
        else:
            buf.append(val)

    result = []
    stack = []
    high_priority = set(["*", "/"])
    low_priority = set(["+", "-"])
    
    for val in buf:
        if(val.isnumeric()):
            result.append(val)
        if(val == "("):
            stack.append(val)
        if(val == ")"):
            while(stack[-1] != "("):
                result.append(stack.pop())
            stack.pop()
        if(val in low_priority):
            while(len(stack) > 0 and stack[-1] in high_priority):
                result.append(stack.pop())
            stack.append(val)
        if(val in high_priority):
            stack.append(val)
            
    while(len(stack) > 0):
        result.append(stack.pop())
    
    return result

def calc(expr):
    operations = {
        "+": lambda x, y: x + y,
        "-": lambda x, y: x - y,
        "*": lambda x, y: x * y,
        "/": lambda x, y: x / y
    }
    
    expr = parse(expr)
    stack = []
    
    for val in expr:
        if(val.isnumeric()):
            stack.append(int(val))
        if(val in operations):
            right = stack.pop()
            left = stack.pop()
            stack.append(operations[val](left, right))
    return stack.pop()

In [27]:
e = '2 * (15 - 3 * 4) - 2'
calc(e)

4